# Discrete-time Fourier Series of a pulse train 

### Introduction
To obtain the frequency transform of a periodic continuous time signal, the Fourier Series is used.  This results in a frequency domain that is sampled.  For a periodic signal that is sampled in the time domain (discrete), then there is an equivalen transform called the Discrete-time Fourier Series (DTFS).  As the signal is periodic in the time domain, then it is sampled in the frequency domain, but also as it is sampled in the time domain, it is periodic in the frequency domain.

The code below defines a pulse shape in the time domain, then uses a transform to obtain its frequency representation.
### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### User specified parameters
The following parameters can be specified. 

Parameter | Meaning
--------- | -------
<code>N</code> | The period of the pulse train (e.g. 36)
<code>L</code> | The number of 1's in the pulse train.  Note this must be an odd. Number for pulse train to be symmetric (e.g. 5)
<code>A</code> | Height of the pulse train (e.g. 1)
<code>figure_cycles</code> | Number of periods to plot the result over(e.g. 3)

In [ ]:
N = 36                 
L = 5                                        
A = 1                  
figure_cycles = 3

### Signal Definition

Now we define the signal.  As it is periodic, we only define one full period, so the number of samples in the signal is <code>N</code>.  For reasons that will become clearer later in the course, we want the signal to be symmetric around the origin, so the length of the pulse must be an odd number of samples.  We will set the first $\frac{L+1}{2}$ samples to the amplitude, <code>A</code>, and the last $\frac{L-1}{2}$ samples to the same values.  Everything in between will be zero.

If you want to define your own signal, you should make sure that it is symmetric across the first sample, or equivalently sample $\frac{N}{2}$.

In [ ]:
last_pulse = (L-1) / 2

# Check for a valid length
if np.floor(last_pulse) != last_pulse :
    raise Exception ('Pulse length, L, must be an odd integer')
    
signal = np.zeros((1, N))
signal[0, 0 : int(last_pulse)+1] = A
signal[0, N - int(last_pulse):] = A

### Compute the DTFS

The transform is usually complex, however as the input was real and symmetric, the transform will also be symmetric and purely real.  The FFT returns one period of the transform, starting with sample 0, so for plotting purposes we will need to copy this.  Note that this step is only required for plotting on the display - in practice as we know it is periodic, we would only use one period of the signal.  Normally this would be the frequency samples from $0$ up to $N-1$.

In [ ]:
transform = np.fft.fft(a=signal)

### Determine the samples to plot

To plot multiple repetitions of signal we first of all define how many repetitions are required, then duplicate the Fourier series that number of times.  We need to keep track of where the centre of the repetitions is, as this will be our origin in the frequency domain.  To get the frequency scale right we work out the frequency terms that we have in the multiple repetitions, then normalise these so that one period of $N$ samples corresponds to the frequency label $2\pi$.

In [ ]:
# Determine the whole number of repetitions required
# Use numpy.ceil function to get the ceiling element-wise
repetitions = np.ceil(figure_cycles/2) * 2

# And duplicate the result that many times
# Use numpy.real function to get the real part of complex number
dtfs = np.tile(np.real(transform),int(repetitions))

extent = np.ceil(N*figure_cycles/2)
centre_index = 1 + N*repetitions/2
offsets = np.arange(-extent, extent+1) 
x_scale = offsets * 2 * np.pi / N
result_indices = centre_index + offsets

### Plot the figure

Now we have the result, for viewing we use matplotlib, and adjust the plot properties to set the correct scales and labels

In [ ]:
# Create the plot figure
plt.figure(figsize = (16, 8))
# Update the label fontsize
plt.rcParams.update({'font.size': 16})

# Plot the amplitude 
(markerLines, stemLines, baseLines) = plt.stem(x_scale,
                                               dtfs[0, result_indices.astype(int)],
                                               use_line_collection = True,
                                               markerfmt ='.')
# Update the size and linewidth of markerlines and stemlines in stem plot 
plt.setp(markerLines, markersize=6)
plt.setp(stemLines, linewidth=1)
plt.setp(baseLines, color = 'black', linewidth=1)  
plt.margins(0.5)

# Set the xlim and ylim of plot
plt.xlim((x_scale[0], x_scale[-1]))
plt.ylim(-A*L*0.3, A*L)

# Formatting the x-axis. 
multiples_of_pi = np.arange(np.ceil(-extent*2/N), np.floor(extent*2/N)+1)
XTicks = np.pi * multiples_of_pi

# Generate the tick labels - it is a bit awkward around -\pi, 0 and \pi,
# so use a dictionary of special cases to do this
xtick_labels = []
special_cases = {
    -1: '$-\pi$',
    0: '0',
    1: '$\pi$'
}
for i in multiples_of_pi:
    xtick_labels = xtick_labels + [special_cases.get(i, '$%d\pi$'%i)]

plt.xticks(XTicks, xtick_labels)   
plt.xlabel('Frequency (rad/sample)')

# Formatting the y-axis. 
plt.ylabel('Amplitude')

# Set the figure title
title = plt.title('L=%d, N=%d, A=%0.0f' %(L, N, A))

### Save the result

If we are not running in a Jupyter notebook, save the resulting plot as a PDF file for use in documentation

In [ ]:
if not is_jupyter():
    plt.savefig('pulse_train_transform.pdf')

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.